# Back Propagation 

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline


In [5]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [6]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [7]:
# build the dataset
block_size = 5  # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
print(n1)
print(n2)
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
print(Xtr[8])
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%


25626
28829
torch.Size([182625, 5]) torch.Size([182625])
tensor([0, 0, 0, 0, 4])
torch.Size([22655, 5]) torch.Size([22655])
torch.Size([22866, 5]) torch.Size([22866])


In [8]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 30 # the dimensionality of the character embedding vectors
n_hidden = 192 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True


35397


In [10]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.8645, grad_fn=<NegBackward0>)

In [12]:
logits.shape

torch.Size([32, 27])

In [13]:
# Now trying to understanding what is happening with dlogprobs
Yb # is the target value
print(logprobs[range(n), Yb].shape)

torch.Size([32])


In [19]:
counts_sum_inv.shape, counts.shape, dcounts.shape, counts_sum.shape

(torch.Size([32, 1]),
 torch.Size([32, 27]),
 torch.Size([32, 27]),
 torch.Size([32, 1]))

In [20]:
logit_maxes.shape, norm_logits.shape, logits.shape, logit_maxes.shape, dlogits2.shape 

(torch.Size([32, 1]),
 torch.Size([32, 27]),
 torch.Size([32, 27]),
 torch.Size([32, 1]),
 torch.Size([32, 27]))

In [21]:
 h.shape, W2.shape, b2.shape

(torch.Size([32, 192]), torch.Size([192, 27]), torch.Size([27]))

In [22]:
bngain.shape, bnraw.shape, bnbias.shape, hpreact.shape

(torch.Size([1, 192]),
 torch.Size([32, 192]),
 torch.Size([1, 192]),
 torch.Size([32, 192]))

In [23]:
bndiff.shape, bnvar_inv.shape

(torch.Size([32, 192]), torch.Size([1, 192]))

In [24]:
dbndiff =(bnvar_inv * dbnraw)
dbndiff.shape

torch.Size([32, 192])

In [25]:
bnvar.shape, bndiff2.shape
dbndiff2 = ((1 / (n-1) )* torch.ones_like(bndiff2)) * dbnvar


In [26]:
bndiff.shape, hprebn.shape, bnmeani.shape 

(torch.Size([32, 192]), torch.Size([32, 192]), torch.Size([1, 192]))

In [27]:
emb.shape, embcat.shape, demb.shape

(torch.Size([32, 5, 30]), torch.Size([32, 150]), torch.Size([32, 5, 30]))

In [28]:
C.shape, Xb.shape

(torch.Size([27, 30]), torch.Size([32, 5]))

In [29]:
C[0], Xb.shape[0], emb[0]

(tensor([ 1.5674, -0.2373, -0.0274, -1.1008,  0.2859, -0.0296, -1.5471,  0.6049,
          0.0791,  0.9046, -0.4713,  0.7868, -0.3284, -0.4330,  1.3729,  2.9334,
          1.5618, -1.6261,  0.6772, -0.8404,  0.9849, -0.1484, -1.4795,  0.4483,
         -0.0707,  2.4968,  2.4448, -0.6701, -1.2199,  0.3031],
        grad_fn=<SelectBackward0>),
 32,
 tensor([[ 1.5674, -0.2373, -0.0274, -1.1008,  0.2859, -0.0296, -1.5471,  0.6049,
           0.0791,  0.9046, -0.4713,  0.7868, -0.3284, -0.4330,  1.3729,  2.9334,
           1.5618, -1.6261,  0.6772, -0.8404,  0.9849, -0.1484, -1.4795,  0.4483,
          -0.0707,  2.4968,  2.4448, -0.6701, -1.2199,  0.3031],
         [ 1.5674, -0.2373, -0.0274, -1.1008,  0.2859, -0.0296, -1.5471,  0.6049,
           0.0791,  0.9046, -0.4713,  0.7868, -0.3284, -0.4330,  1.3729,  2.9334,
           1.5618, -1.6261,  0.6772, -0.8404,  0.9849, -0.1484, -1.4795,  0.4483,
          -0.0707,  2.4968,  2.4448, -0.6701, -1.2199,  0.3031],
         [-0.4805,  0.5331,  0

In [30]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -(1/n)
cmp("logprobs",dlogprobs, logprobs)

dprobs = (1/probs) * dlogprobs
cmp("probs",dprobs, probs)

dcounts_sum_inv =  (counts * dprobs).sum(1, keepdims =True)
cmp("counts_sum_inv",dcounts_sum_inv, counts_sum_inv)

dcounts = counts_sum_inv *  dprobs  # No .sum() because there is no broad casting going on ..... 

dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp("counts_sum",dcounts_sum, counts_sum)

dcounts += torch.ones_like(counts) * dcounts_sum
cmp("counts",dcounts, counts)

dnorm_logits = counts * dcounts
cmp("norm_logits",dnorm_logits, norm_logits)

dlogit_maxes = ((- 1) * dnorm_logits).sum(1, keepdims = True)
cmp("logit_maxes",dlogit_maxes, logit_maxes)

dlogits = dnorm_logits
temp = logits.max(1).indices
dlogits2 = torch.zeros_like(logits)
dlogits2[range(n), temp] = torch.reshape(dlogit_maxes, (1,32))
dlogits += dlogits2
cmp("logits",dlogits, logits)

dh = dlogits @ W2.T
dW2 =  h.T @ dlogits
db2 = dlogits.sum(0)

cmp("h",dh, h)
cmp("W2",dW2, W2)
cmp("b2",db2, b2)

dhpreact = (1- h**2) * dh
cmp("hpreact",dhpreact, hpreact)

dbnraw = bngain * dhpreact
cmp("bnraw",dbnraw, bnraw)

dbngain = (bnraw * dhpreact).sum(0,keepdims =True)
cmp("bngain",dbngain, bngain)

dbnbias = (dhpreact).sum(0,keepdims =True)
cmp("bnbias",dbnbias, bnbias)

dbnvar_inv =(bndiff * dbnraw).sum(0, keepdims = True)
cmp("dnvar_inv",dbnvar_inv, bnvar_inv)

dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
cmp("bnvar",dbnvar, bnvar) 

dbndiff2 = ((1 / (n-1) )* torch.ones_like(bndiff2)) * dbnvar
cmp("bndiff2",dbndiff2, bndiff2) 

dbndiff =(bnvar_inv * dbnraw)
dbndiff += (2*(bndiff)) * dbndiff2
cmp("bndiff",dbndiff, bndiff) 

dhprebn = dbndiff  #intermediate


dbnmeani = (-1 * dbndiff).sum(0, keepdims= True)
cmp("bnmeani",dbnmeani, bnmeani)

dhprebn += (((1 / n) *torch.ones_like(hprebn)) * dbnmeani)
cmp("hprebn",dhprebn, hprebn)

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

cmp("dembcat",dembcat, embcat)
cmp("W1",dW1, W1)
cmp("b1",db1, b1)

demb = dembcat.view(emb.shape)
cmp("emb",demb, emb)

dC = torch.zeros_like(C)

for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i,j]
        dC[ix] += demb[i,j]
cmp("C",dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff:

In [31]:
#Skipping the steps of finding derivative of loss function


dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n
cmp("logits",dlogits, logits)



logits          | exact: False | approximate: True  | maxdiff: 7.2177499532699585e-09


In [32]:
#Skipping the unnecessary steps of finding derivative of batch-normalization

dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) 


hprebn          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09


In [38]:
# Building a neural net withou loss.backward()

# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 30 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    #loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n
    # 2nd layer backprop
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    # tanh
    dhpreact = (1.0 - h**2) * dh
    # batchnorm backprop
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    #if i >= 100: # TODO: delete early breaking when you're ready to train the full net
       #break

36837
      0/ 200000: 3.4033
  10000/ 200000: 2.0024
  20000/ 200000: 2.1535
  30000/ 200000: 1.9246
  40000/ 200000: 2.1613
  50000/ 200000: 2.1817
  60000/ 200000: 1.7134
  70000/ 200000: 2.1214
  80000/ 200000: 2.3109
  90000/ 200000: 2.0172
 100000/ 200000: 2.0472
 110000/ 200000: 1.9237
 120000/ 200000: 1.7872
 130000/ 200000: 2.3727
 140000/ 200000: 1.6427
 150000/ 200000: 2.0260
 160000/ 200000: 1.6802
 170000/ 200000: 2.4505
 180000/ 200000: 1.9378
 190000/ 200000: 1.7813


In [39]:
with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [40]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 1.8787823915481567
val 2.0106563568115234


In [41]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmah.
amell.
kitzirre.
reet.
khalani.
emmahnee.
deliah.
jareli.
nelonia.
chaiir.
kaleigh.
hams.
kinder.
juliani.
rayvin.
quintelo.
dearyn.
kailee.
dusti.
eddeci.
